# S&P 500 Analysis

## Import the Packages, Functions and Data

In [1]:
import pandas as pd
from DQN_functions import create_states, ReplayMemory, DQNAgent,train_agent,evaluate_agent, ConvDQN,plot_training,create_action_episode_df
import pandas as pd
import torch
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_pickle('data/SP500.pkl')

## Outlier Removal 


In [3]:
df['daily_return'] = df['Close'].pct_change()
df['daily_return'] = df['daily_return'].fillna(0)
#mean plus 3sigma value for absolute daily returns
mean = df['daily_return'].mean()
std = df['daily_return'].std()
upper_limit = mean + 2*std
lower_limit = mean - 2*std
print(f'Mean: {mean}, Std: {std}, Upper Limit: {upper_limit}, Lower Limit: {lower_limit}')
#number of rows that are outside the 3 sigma range
print(f'Number of rows outside 3 sigma range: {len(df[(df["daily_return"]>upper_limit) | (df["daily_return"]<lower_limit)])}')
print(f'Percentage of rows outside 3 sigma range: {len(df[(df["daily_return"]>upper_limit) | (df["daily_return"]<lower_limit)])/len(df)*100}% ')

#if daily return is outside 3 sigma range, replace all the values with previous day's values
for i in range(1,len(df)):
    if df['daily_return'][i]>upper_limit or df['daily_return'][i]<lower_limit:
        df['Close'][i] = df['Close'][i-1]
        df['Open'][i] = df['Open'][i-1]
        df['High'][i] = df['High'][i-1]
        df['Low'][i] = df['Low'][i-1]
        df['Volume'][i] = df['Volume'][i-1]
        df['daily_return'][i] = 'NaN'

Mean: 0.00037873998112499885, Std: 0.012040180200659603, Upper Limit: 0.024459100382444206, Lower Limit: -0.023701620420194205
Number of rows outside 3 sigma range: 244
Percentage of rows outside 3 sigma range: 4.848003179018478% 


## DQN

### Training

In [4]:
df_base = df[['Open','High','Low','Close','Volume']]

#first 80% of the data is train
df_train = df_base.iloc[:int(len(df_base)*0.8)]
#last 20% of the data is test
df_test = df_base.iloc[int(len(df_base)*0.8):]

In [5]:
display(df_train.head())

display(df_test.head())

,Open,High,Low,Close,Volume
Date,,,,,
2004-03-31,1127.000000,1130.829956,1121.459961,1126.209961,1560700000
2004-04-01,1126.209961,1135.670044,1126.199951,1132.170044,1560700000
2004-04-02,1132.170044,1144.810059,1132.170044,1141.810059,1629200000
2004-04-05,1141.810059,1150.569946,1141.640015,1150.569946,1413700000
2004-04-06,1150.569946,1150.569946,1143.300049,1148.160034,1397700000


,Open,High,Low,Close,Volume
Date,,,,,
2020-03-30,2457.770020,2571.419922,2407.530029,2475.560059,8300010000
2020-03-31,2614.689941,2641.389893,2571.149902,2584.590088,6576210000
2020-04-01,2614.689941,2641.389893,2571.149902,2584.590088,6576210000
2020-04-02,2458.540039,2533.219971,2455.790039,2526.899902,6464190000
2020-04-03,2514.919922,2538.179932,2459.959961,2488.649902,6096970000


In [6]:
print(df_train.shape, df_test.shape)
print(df_train.shape[0] + df_test.shape[0])
print(df.shape[0])

(4026, 5) (1007, 5)
5033
5033


In [7]:
window_size = 26
states = create_states(df_train, window_size)
test_states = create_states(df_test, window_size)
input_dim_conv = states.shape[2]
output_dim = 3
print("Shape of states:", states.shape)

Shape of states: (4000, 26, 5)


In [8]:
model = ConvDQN(input_dim_conv, output_dim, window_size)
memory = ReplayMemory(50000)
agent = DQNAgent(input_dim_conv, output_dim, window_size,model, lr=0.0001, gamma=0.95, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.9995)

In [9]:
%%time
log_train = train_agent(agent, states, 350, batch_size=32, price_history=df_train[window_size:])

Episode 1/350, Total Reward: 4005.8183410207357, Loss: 25.438297878296574
Episode 2/350, Total Reward: 4005.7189363697157, Loss: 25.139787896624963
Episode 3/350, Total Reward: 4005.6348669239155, Loss: 23.865182061650245
Episode 4/350, Total Reward: 4005.6145298458064, Loss: 23.11789382598156
Episode 5/350, Total Reward: 4005.31621782944, Loss: 20.792510486867844
Episode 6/350, Total Reward: 4005.5585361000453, Loss: 18.83726608576915
Episode 7/350, Total Reward: 4006.1887506803355, Loss: 17.204917234990504
Episode 8/350, Total Reward: 4005.3652140762174, Loss: 14.106568426000965
Episode 9/350, Total Reward: 4005.54487705279, Loss: 14.951967453128654
Episode 10/350, Total Reward: 4005.239732591109, Loss: 10.10887716561938
Episode 11/350, Total Reward: 4005.8470920317845, Loss: 5.797340706396096
Episode 12/350, Total Reward: 4005.4167434995948, Loss: 0.010171719305781673
Episode 13/350, Total Reward: 4005.410362286636, Loss: -0.022895269702831435
Episode 14/350, Total Reward: 4005.1126

In [ ]:
log_train.to_csv('SP500/log_train_CNN_adjusted_lr_hyp_tune_mu2sigma_penalised_hold.csv', index=False)

In [ ]:
#save the model
torch.save(agent.model.state_dict(), 'SP500/dqn_model_log_train_CNN_adjusted_lr_hyp_tuned_scaled.pth')


### Evaluating and Testing

In [ ]:
log_train = pd.read_csv('SP500/log_train_CNN_adjusted learning rate.csv')

In [ ]:
plot_training(log_train)

In [ ]:
last_episode = log_train[log_train['Episode'] == log_train['Episode'].max()]

In [ ]:
last_episode.reset_index(drop=True, inplace=True)
last_episode.head()

In [ ]:
last_episode['Close'] = last_episode['Price'].to_list()

In [ ]:
last_episode['Capital_DQN'] = capital_calculation(last_episode,'Action')

In [ ]:
action_episode_df = create_action_episode_df(log_train)

In [ ]:
#get value counts of all columns in the DataFrame
action_episode_df.apply(pd.Series.value_counts).transpose()[['Buy']].plot()

In [ ]:
# Initialize the agent
test_state_size = test_states.shape[1] * test_states.shape[2]
action_size = 3
agent = DQNAgent(state_size, action_size, lr=0.00001, gamma=0.95, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.999)

In [ ]:
test_state_size

In [ ]:
model_path = 'SP500/dqn_modelSP500_CNN_500_wo_outliers_hyp_tuned.pth'
agent.model.load_state_dict(torch.load(model_path))

# Ensure the model is in evaluation mode
agent.model.eval()

##